# NLP with Ensemble Method

Les scores obtenus avec la regression logistique pour les différentes catégories étaient les suivants :
- f1 score pour la category_1 : 0.1601514910747391
- f1 score pour la category_2 : 0.6212450536216092
- f1 score pour la category_3 : 0.7704773846842813
- f1 score pour la category_4 : 0.0

Là où la regression logistique semble assez bien performée sur la catégorie 3, elle n'est d'aucune utilité pour prédire la catégorie 4. Nous allons donc essayer d'améliorer ces résultats en utilisant d'autres méthodes et prendre le meilleur classificateur pour chaque catégorie

### Extraction des données

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

DIRECTORY = "challenge_dataset/"

# extract data
X = pd.read_csv(DIRECTORY + 'X_train_clean.csv', sep=';')
X = X['lemmas'].apply(lambda x: ' '.join(eval(x))).to_list()

y = pd.read_csv(DIRECTORY + 'y_train.csv', sep=';').drop(columns=['Id'])
categories = y.columns

## Naives Bayes

In [55]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import MultinomialNB

n_splits = 5 # nombre de splits pour le KFold
NB = make_pipeline(CountVectorizer(),
    OneVsRestClassifier(MultinomialNB())
)
cv = KFold(n_splits, shuffle=True)
for category in categories:
    print("f1 score", category, ":", np.mean(cross_val_score(NB, X, y[category], cv=cv, scoring='f1')))

f1 score category_1 : 0.5936116542073988
f1 score category_2 : 0.7054110010631749
f1 score category_3 : 0.7763704947269401
f1 score category_4 : 0.07076923076923078


## Linear SVC

In [56]:
from sklearn.svm import LinearSVC

n_splits = 5 # nombre de splits pour le KFold
SVC = make_pipeline(CountVectorizer(),
    OneVsRestClassifier(LinearSVC())
)
cv = KFold(n_splits, shuffle=True)
for category in categories:
    print("f1 score", category, ":", np.mean(cross_val_score(SVC, X, y[category], cv=cv, scoring='f1')))

f1 score category_1 : 0.5563002520965169
f1 score category_2 : 0.7411318048640563
f1 score category_3 : 0.7762927300948628
f1 score category_4 : 0.29358690844758956


## Logistic Regression

In [57]:
from sklearn.linear_model import LogisticRegression

n_splits = 5 # nombre de splits pour le KFold
LR = make_pipeline(CountVectorizer(),
    OneVsRestClassifier(LogisticRegression(solver='sag', max_iter=1000))
)
cv = KFold(n_splits, shuffle=True)
for category in categories:
    print("f1 score", category, ":", np.mean(cross_val_score(LR, X, y[category], cv=cv, scoring='f1')))

f1 score category_1 : 0.5357894736842106
f1 score category_2 : 0.7402792408320991
f1 score category_3 : 0.7801076355661307
f1 score category_4 : 0.12380952380952381


Le meilleur classificateur semble être le Linear SVC

## Prédiction pour Kaggle

In [60]:
X_test = pd.read_csv(DIRECTORY + 'X_test_clean.csv', sep=';')
index = X_test['Id']
X_test = X_test['lemmas'].apply(lambda x: ' '.join(eval(x))).to_list()

y_pred = pd.DataFrame(index, columns=['Id'])
for category in categories:
    SVC.fit(X, y[category])
    y_pred[category] = SVC.predict(X_test)
y_pred

,Id,category_1,category_2,category_3,category_4
0,599,0,0,0,0
1,600,0,0,0,0
2,602,0,0,0,0
3,603,0,1,0,0
4,604,0,0,1,0
...,...,...,...,...,...
152,798,0,0,1,0
153,799,0,0,1,0
154,800,1,0,0,0
155,801,0,1,0,0


In [62]:
y_pred.to_csv("results/y_linear_SVC_on_labels_076646.csv", index=False)